In [1]:
# tensorflow dependencies 
import logging
import time

import numpy as np
#import matplotlib.pyplot as plt


#import tensorflow_datasets as tfds ==> not required cause building custom datatset
import tensorflow as tf

import tensorflow_text

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
preprocessor = hub.KerasLayer("MuRIL_preprocess_1")

In [ ]:
encoder = hub.KerasLayer("MuRIL_1", trainable=True)

# Encoder Output

In [ ]:
sentences = ["This is a text", "This is another text"]
list_tensor = tf.convert_to_tensor(sentences)

output = encoder(preprocessor(list_tensor))

print(output["sequence_output"])
print(output["sequence_output"].shape)

# Decoder Output

In [ ]:
import torch

In [ ]:
tokens = preprocessor(list_tensor)['input_word_ids']


for i in range(tokens.shape[0]):
    token = np.array(tokens[i])
    token_list = []
    for _ in token:
        if _ != 0:
            token_list.append(_)
    
    input_tokens = token_list[:-1]
    input_labels = token_list[1:]
    
    for j in range(128):
        input_tokens.append(0)
        input_labels.append(0)
        
    input_labels = tf.ragged.constant([input_labels[:128]]).to_tensor()
    input_tokens = tf.ragged.constant([input_tokens[:128]]).to_tensor()
    
    if i == 0:
        decoder_label = input_labels
        decoder_input = input_tokens
        
    else:
        decoder_label = tf.concat([decoder_label, input_labels], axis = 0)
        decoder_input = tf.concat([decoder_input, input_tokens], axis = 0)    

In [ ]:
decoder_input.shape

In [ ]:
decoder_label.shape

In [28]:
class Encoder():
    def __init__(self):
        self.encoder = hub.KerasLayer("MuRIL_1", trainable=True)
        self.preprocessor = hub.KerasLayer("MuRIL_preprocess_1")
        
    def generate_tokens(self, sentences):
        #list_tensor = tf.convert_to_tensor(sentences)
        #processor_output = self.preprocessor(list_tensor)
        processor_output = self.preprocessor(tf.constant([sentences]))
        encoder_output = self.encoder(processor_output)
        
        return encoder_output["sequence_output"]

In [ ]:
sentences = ['This is a text', 
             'This is a text in hindi', 
             'This is a text in hindi']

encoder = Encoder()
encoder_output = encoder.generate_tokens(sentences)

In [ ]:
encoder_output.shape

In [31]:
class DecoderLabelsInputs():
    def __init__(self):
        self.preprocessor = hub.KerasLayer("MuRIL_preprocess_1")
        
    def generate_labels_inputs(self, sentences):
        #list_tensor = tf.convert_to_tensor(sentences)
        #tokens = preprocessor(list_tensor)['input_word_ids']
        
        tokens = self.preprocessor(tf.constant([sentences]))['input_word_ids']
        
        for i in range(tokens.shape[0]):
            token = np.array(tokens[i])
            token_list = []
            for _ in token:
                if _ != 0:
                    token_list.append(_)

            input_tokens = token_list[:-1]
            input_labels = token_list[1:]

            for j in range(128):
                input_tokens.append(0)
                input_labels.append(0)

            input_labels = tf.ragged.constant([input_labels[:128]]).to_tensor()
            input_tokens = tf.ragged.constant([input_tokens[:128]]).to_tensor()

            if i == 0:
                decoder_label = input_labels
                decoder_input = input_tokens

            else:
                decoder_label = tf.concat([decoder_label, input_labels], axis = 0)
                decoder_input = tf.concat([decoder_input, input_tokens], axis = 0)
                
        
        return decoder_label, decoder_input

In [ ]:
sentences = ['This is a text', 
             'This is a text in hindi', 
             'This is a text in hindi']

decoder_labels_inputs = DecoderLabelsInputs()
decoder_label, decoder_input = decoder_labels_inputs.generate_labels_inputs(sentences)

# Attentions

In [5]:
def positional_encoding(length, depth):
    depth = depth/2
    positions = np.arange(length)[:, np.newaxis]
    depths = np.arange(depth)[np.newaxis, :]/depth
    
    angle_rates = 1 / (10000**depths)
    angle_rads = positions * angle_rates
    
    pos_encoding = np.concatenate(
        [np.sin(angle_rads), np.cos(angle_rads)],
        axis = -1)
    
    return tf.cast(pos_encoding, dtype = tf.float32)

In [6]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero = True)
        self.pos_encoding = positional_encoding(length = 2048, depth = d_model)
        
    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)
    
    def call(self, x):
        length = tf.shape(x)[1]
        x = self.embedding(x)
        
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x = x + self.pos_encoding[tf.newaxis, :length, :]
        return x

In [7]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=8, key_dim=768)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

In [8]:
class CausalMask():
    def __init__(self):
        super().__init__()
        
    def _compute_causal_mask(self, query, value=None):    
        q_seq_length = tf.shape(query)[1]
        v_seq_length = q_seq_length if value is None else tf.shape(value)[1]
        return tf.linalg.band_part(tf.ones((1, q_seq_length, v_seq_length), tf.bool), -1, 0)

    def _compute_attention_mask(self, query, value, key=None, attention_mask=None, use_causal_mask=False):
        query_mask = getattr(query, "_keras_mask", None)
        value_mask = getattr(value, "_keras_mask", None)
        key_mask = getattr(key, "_keras_mask", None)
        auto_mask = None
        if query_mask is not None:
            query_mask = tf.cast(query_mask, tf.bool)  # defensive casting
            # B = batch size, T = max query length
            auto_mask = query_mask[:, :, tf.newaxis]  # shape is [B, T, 1]
        if value_mask is not None:
            value_mask = tf.cast(value_mask, tf.bool)  # defensive casting
            # B = batch size, S == max value length
            mask = value_mask[:, tf.newaxis, :]  # shape is [B, 1, S]
            auto_mask = mask if auto_mask is None else auto_mask & mask
        if key_mask is not None:
            key_mask = tf.cast(key_mask, tf.bool)  # defensive casting
            # B == batch size, S == max key length == max value length
            mask = key_mask[:, tf.newaxis, :]  # shape is [B, 1, S]
            auto_mask = mask if auto_mask is None else auto_mask & mask
        if use_causal_mask:
            # the shape of the causal mask is [1, T, S]
            mask = self._compute_causal_mask(query, value)
            auto_mask = mask if auto_mask is None else auto_mask & mask
        if auto_mask is not None:
            # merge attention_mask & automatic mask, to shape [B, T, S]
            attention_mask = (
                auto_mask
                if attention_mask is None
                else tf.cast(attention_mask, bool) & auto_mask
            )
        return attention_mask

In [9]:
class CrossAttention(BaseAttention, CausalMask):
    def call(self, x, context):
        mask = CausalMask()
        mask = mask._compute_attention_mask(query=x, 
                                        value=context, 
                                        key=context, 
                                        use_causal_mask=False)
        
        attn_output = self.mha(query = x,
                                key = context,
                                value = context,
                                attention_mask = mask)
        
        #self.last_attn_scores = attn_scores
        
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        
        return x

In [10]:
class CausalAttention(BaseAttention):
    def call(self, x):
        mask = CausalMask()
        mask = mask._compute_attention_mask(query=x, 
                                        value=x, 
                                        key=x, 
                                        use_causal_mask=True)
        
        attn_output = self.mha(query = x,
                                key = x,
                                value = x,
                                attention_mask = mask)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        
        return x

In [11]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model = 768, dff = 2048, dropout_rate = 0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation = 'relu'),
            tf.keras.layers.Dense(d_model),
            tf.keras.layers.Dropout(dropout_rate)
        ])
        
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()
        
    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x)
        
        return x

In [12]:
class DecoderLayer(tf.keras.layers.Layer):      
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.causal_self_attention = CausalAttention()
        self.cross_attention = CrossAttention()
        self.ffn = FeedForward()
        
    
    def call(self, x, context):
        x = self.causal_self_attention(x = x)
        x = self.cross_attention(x = x, context = context)
        
        x = self.ffn(x)
        return x

In [13]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers):
        super(Decoder, self).__init__()
        
        self.num_layers = num_layers
        self.pos_embedding = PositionalEmbedding(vocab_size = 197285, 
                                                 d_model = 768)
        self.dropout = tf.keras.layers.Dropout(0.1)
        
        self.dec_layers = [
            DecoderLayer()
            for _ in range(num_layers)
        ]
        
    def call(self, x, context):
        x = self.pos_embedding(x)
        x = self.dropout(x)
        
        for i in range(self.num_layers):
            x = self.dec_layers[i](x, context)
            
        return x

In [14]:
class TransformerDecoder(tf.keras.Model):
    def __init__(self):
        super().__init__()        
        self.decoder = Decoder(num_layers = 4)
        self.final_layer = tf.keras.layers.Dense(197285)
    
        
    def call(self, inputs):
        encoder_output, decoder_input = inputs
 
        decoder_output = self.decoder(decoder_input, encoder_output)
        
        logits = self.final_layer(decoder_output)
        return logits

In [15]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self):
        super().__init__()
        self.d_model = 768
        self.d_model = tf.cast(self.d_model, tf.float32)
        
        self.warmup_steps = 4000
        
    def __call__(self, step):
        step = tf.cast(step, dtype = tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [16]:
learning_rate = CustomSchedule()
optimizer = tf.keras.optimizers.Adam(learning_rate, 
                                     beta_1 = 0.9,
                                     beta_2 = 0.98,
                                     epsilon = 1e-9)

In [17]:
def masked_loss(decoder_label, pred_label):
    mask = decoder_label != 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
                                            from_logits = True,
                                            reduction = 'none')
    
    loss = loss_object(decoder_label, pred_label)
    
    mask = tf.cast(mask, dtype = loss.dtype)
    
    loss *= mask
    
    loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
    return loss

In [18]:
def masked_accuracy(decoder_label, pred_label):
    pred_label = tf.argmax(pred_label, axis = 2)
    decoder_label = tf.cast(decoder_label, pred_label.dtype)
    
    match = decoder_label == pred_label
    mask = decoder_label != 0
    
    match = match & mask
    
    match = tf.cast(match, dtype = tf.float32)
    mask = tf.cast(mask, dtype = tf.float32)
    
    return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [19]:
sample_transformer_decoder = TransformerDecoder()
sample_transformer_decoder.compile(loss = masked_loss,
                          optimizer = optimizer,
                          metrics = [masked_accuracy])

In [ ]:
sample_transformer_decoder.fit(x = (encoder_output, decoder_input),
                               y = decoder_label,
                               epochs = 5,
                               verbose = 1)

In [ ]:
#Ensure internet connectivity

from datasets import load_dataset
from tqdm import tqdm

english_sent = []
hindi_sent = []

try:
    dataset = load_dataset("cfilt/iitb-english-hindi")
    list_dataset = dataset['train']['translation']

    for i in tqdm(range(len(list_dataset))):
        english_sent.append(list_dataset[i]['en'])
        hindi_sent.append(list_dataset[i]['hi'])

except ConnectionError:
    english_sent.append('This is an english sent')
    hindi_sent.append('This is a hindi text')

In [ ]:
print(english_sent[10000])
print(hindi_sent[10000])

In [ ]:
len(english_sent)

In [ ]:
english_sent = english_sent[:1000]
hindi_sent = hindi_sent[:1000]

encoder = Encoder()
encoder_output = encoder.generate_tokens(hindi_sent)

decoder_labels_inputs = DecoderLabelsInputs()
decoder_label, decoder_input = decoder_labels_inputs.generate_labels_inputs(english_sent)

In [ ]:
sample_transformer_decoder.fit(x = (encoder_output, decoder_input),
                               y = decoder_label,
                               epochs = 5,
                               verbose = 1)

In [20]:
import docx

In [21]:
def process_stories():
    full_text_hindi = []
    full_text_english = []
    
    #process hindi
    for i in range(1,6):
        file_name = "hindi_stories/hindi/S" + str(i) + "-HINDI.docx"
        doc = docx.Document(file_name)
        for para in doc.paragraphs:
            full_text_hindi.append(para.text)
            
    #process english
    for i in range(1,6):
        file_name = "hindi_stories/english/S" + str(i) + "-ENG.docx"
        doc = docx.Document(file_name)
        for para in doc.paragraphs:
            full_text_english.append(para.text)
    
    
    return full_text_hindi, full_text_english

In [22]:
hindi_text, english_text = process_stories()

In [ ]:
print(len(hindi_text))
print(len(english_text))

In [ ]:
english_sent = english_text
hindi_sent = hindi_text

encoder = Encoder()
encoder_output = encoder.generate_tokens(hindi_sent)

decoder_labels_inputs = DecoderLabelsInputs()
decoder_label, decoder_input = decoder_labels_inputs.generate_labels_inputs(english_sent)

In [32]:
from tqdm import tqdm

encoder_output_list = []
decoder_input_list = []
decoder_label_list = []

english_sent = english_text
hindi_sent = hindi_text

encoder = Encoder()
decoder_labels_inputs = DecoderLabelsInputs()

for i in tqdm(range(len(hindi_sent))):
    encoder_output = encoder.generate_tokens(hindi_sent[i])
    encoder_output_list.append(encoder_output)
    
    decoder_label, decoder_input = decoder_labels_inputs.generate_labels_inputs(english_sent[i])
    decoder_input_list.append(decoder_input)
    decoder_label_list.append(decoder_label)

100%|███████████████████████████████████████| 6149/6149 [14:33<00:00,  7.04it/s]


In [33]:
tf_1 = encoder_output_list[0]
tf_2 = encoder_output_list[1]

encoder_output = tf.concat([tf_1, tf_2], axis = 0)

for i in tqdm(range(len(encoder_output_list[2:]))):
    tf_2 = encoder_output_list[i]
    encoder_output = tf.concat([encoder_output, tf_2], axis = 0)

100%|███████████████████████████████████████| 6147/6147 [09:17<00:00, 11.03it/s]


In [34]:
tf_1 = decoder_input_list[0]
tf_2 = decoder_input_list[1]

decoder_input = tf.concat([tf_1, tf_2], axis = 0)

for i in tqdm(range(len(decoder_input_list[2:]))):
    tf_2 = decoder_input_list[i]
    decoder_input = tf.concat([decoder_input, tf_2], axis = 0)

100%|████████████████████████████████████| 6147/6147 [00:00<00:00, 11946.85it/s]


In [35]:
tf_1 = decoder_label_list[0]
tf_2 = decoder_label_list[1]

decoder_label = tf.concat([tf_1, tf_2], axis = 0)

for i in tqdm(range(len(decoder_label_list[2:]))):
    tf_2 = decoder_label_list[i]
    decoder_label = tf.concat([decoder_label, tf_2], axis = 0)

100%|████████████████████████████████████| 6147/6147 [00:00<00:00, 12111.75it/s]


In [36]:
sample_transformer_decoder.fit(x = (encoder_output, decoder_input),
                               y = decoder_label,
                               epochs = 20,
                               verbose = 1)

Epoch 1/20
193/193 [==============================] - 10389s 54s/step - loss: 1.2680 - masked_accuracy: 0.0578
Epoch 2/20
193/193 [==============================] - 10631s 55s/step - loss: 0.7923 - masked_accuracy: 0.0691
Epoch 3/20
193/193 [==============================] - 11619s 60s/step - loss: 0.6517 - masked_accuracy: 0.1404
Epoch 4/20
193/193 [==============================] - 12649s 66s/step - loss: 0.5625 - masked_accuracy: 0.2296
Epoch 5/20
193/193 [==============================] - 12709s 66s/step - loss: 0.5010 - masked_accuracy: 0.2796
Epoch 6/20
193/193 [==============================] - 12702s 66s/step - loss: 0.4533 - masked_accuracy: 0.3229
Epoch 7/20
193/193 [==============================] - 12757s 66s/step - loss: 0.4140 - masked_accuracy: 0.3605
Epoch 8/20
193/193 [==============================] - 12832s 66s/step - loss: 0.3785 - masked_accuracy: 0.3964
Epoch 9/20
193/193 [==============================] - 12824s 66s/step - loss: 0.3447 - masked_accuracy: 0.4329
E

KeyboardInterrupt: 

# Prediction

In [68]:
hindi_text = 'कुछ जगहों पर अंग्रेजी शब्दों के हिन्दी अर्थ के रूप में रखे गये हैं'
encoder_output_pred = encoder.generate_tokens(hindi_text)

english_text = 'At few places english word hindi meaning are kept'
decoder_label_pred, decoder_input_pred = decoder_labels_inputs.generate_labels_inputs(english_text)

In [69]:
predictions = sample_transformer_decoder([encoder_output_pred, decoder_input_pred], training=False)

In [70]:
predictions

<tf.Tensor: shape=(1, 128, 197285), dtype=float32, numpy=
array([[[ -9.316858,  -9.341396,  -9.314201, ...,  -9.29075 ,
          -9.302386,  -9.311396],
        [-11.013479, -10.98209 , -11.04376 , ..., -11.015772,
         -10.946828, -11.008596],
        [-13.263971, -13.300353, -13.421228, ..., -13.271173,
         -13.264562, -13.373016],
        ...,
        [-14.204481, -14.176577, -14.23652 , ..., -14.224231,
         -14.265865, -14.266887],
        [-14.167075, -14.140238, -14.201182, ..., -14.184856,
         -14.229596, -14.228021],
        [-14.186508, -14.158877, -14.224222, ..., -14.203712,
         -14.248528, -14.245734]]], dtype=float32)>

In [71]:
masked_accuracy(pred_label=predictions, decoder_label=decoder_label_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>